In [ ]:
from load_data import loading_and_pre_processing_pipeline
import os

feature_data = loading_and_pre_processing_pipeline().sort_values("user_id")

try:
    limit = os.environ["LIMIT"]
except KeyError:
    limit = None

if limit is not None:
    feature_data = feature_data.head(n=limit)

cleaned_feature_data = feature_data.dropna(axis=0)

print(
    f'{len(feature_data)} records from {len(set(cleaned_feature_data["user_id"].values))} users present. '
)
print(f"{len(cleaned_feature_data)} of which are complete.")
cleaned_feature_data.head()

In [ ]:
feature_data.info()

In [ ]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

from sklearn.linear_model import LogisticRegression
from typing import List

from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector

from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, PolynomialFeatures
from sklearn.pipeline import make_pipeline


def train_model(features: List[str], target: str, data: pd.DataFrame, random_state=42):

    working_data = data.dropna(subset=[target])

    # Do train test split along user ids meaning the data of one group of users is the test set and the
    # data of another group of users is the training set and data of one particular user can not be in both.
    gsp = GroupShuffleSplit(test_size=0.33, n_splits=1, random_state=random_state)
    split = gsp.split(working_data, groups=working_data["user_id"])
    train, test = next(split)

    training_data = working_data.iloc[train]
    testing_data = working_data.iloc[test]

    X = training_data[features]
    y = training_data[target].astype(int)

    # pipe = make_pipeline(StandardScaler(), LogisticRegression())
    # impute missing values: numeric values with their mean and boolean values (reported symptoms) with False
    num_pipe = make_pipeline(
        StandardScaler(), SimpleImputer(strategy="mean", add_indicator=True)
    )
    cat_pipe = make_pipeline(
        SimpleImputer(strategy="constant", fill_value=False),
        OrdinalEncoder(),
    )
    preprocessor_linear = make_column_transformer(
        [num_pipe, selector(dtype_include="number")],
        [cat_pipe, selector(dtype_include="object")],
        n_jobs=2,
    )

    # use polynomial features to enable the model to fit different sets of parameters for different virus variants
    # Also use class_weight='balanced' in logistic regression to correct for imbalance between positive and negative test results.
    pipe = make_pipeline_with_sampler(
        preprocessor_linear,
        PolynomialFeatures(interaction_only=True),
        LogisticRegression(max_iter=1000, class_weight="balanced"),
    )

    pipe.fit(X, y)

    return (
        pipe,
        testing_data[features],
        testing_data[target].astype(int),
        test,
        list(set(list(training_data["user_id"].values))),
    )

In [ ]:
from analysis_plots import plot_analysis

feature_names = [
    "rhr_metric",
    "steps_metric",
    "sleep_duration_metric",
    "days_since_last_dose",
    "vaccination_status",
    "fittness",
    "sex",
    "age",
    "omicronba1_share",
    "omicronba2_share",
    "delta_share",
    "chills",
    "body_pain",
    "loss_of_taste_and_smell",
    "fatigue",
    "cough",
    "cold",
    "diarrhea",
    "sore_throat",
]
target = "test_result"

pipeline, X_test, y_test, _, __ = train_model(feature_names, target, feature_data)

fig = plot_analysis(pipeline, X_test, y_test, feature_names, 0.5)

fig.savefig("model_metrics.png", dpi=300)

In [ ]:
import ramda as R
from datetime import timedelta
from numpy import random

MEAN_DAYS = 7


@R.curry
def predicted_incidence(classifier, feature_data, feature_names, threshold):
    data = feature_data.set_index(["user_id", "date"])

    # predicted probability for a set of vital data and reported symptoms to represent an infection
    proba_infected = classifier.predict_proba(data[feature_names]).T[1].T

    p = pd.DataFrame(data=proba_infected, index=data.index, columns=["probability"])

    colname = f"{MEAN_DAYS}d rolling mean detections"

    # threshold predicted probabilities to classify data points in positive and negative predictions
    p[colname] = p["probability"] > threshold

    p["all"] = p["probability"] > 0

    # if individuals are detected as infected in more than one week, take only the first detection
    only_first_positive_detections = (
        p[p[colname] == True]
        .reset_index()
        .sort_values(["user_id", "date"])
        .groupby(["user_id"])
        .agg({colname: "first", "all": "first", "date": lambda x: list(x)[0]})
        .reset_index()
    )
    all_negative_detections = p[p[colname] == False].reset_index()

    detected_infections = pd.concat(
        [only_first_positive_detections, all_negative_detections]
    )

    # since the date attached to each data point is the first day of the week, but the test
    # might have taken place at any day during the week, we add a random number
    # between 1 and 7 days to the date
    detected_infections["date"] = detected_infections["date"].apply(
        lambda d: d + timedelta(days=int(random.randint(0, 7, 1)))
    )

    # Sum detections per day, then take the rolling 7 day mean
    rolling_mean_detections = (
        detected_infections.groupby("date")
        .sum()
        .rolling(f"{MEAN_DAYS}D")
        .mean()
        .drop(columns=["probability", "user_id"])
    )

    # devide rolling 7 day mean of detections by rolling 7 day mean of all data points
    # and multiply by 100.000 to get the 7 day incidence from the nowcast.
    rolling_mean_detections[colname] = (
        rolling_mean_detections[colname].div(rolling_mean_detections["all"], axis=0)
        * 100_000
    )

    return rolling_mean_detections.reset_index(), colname


def predict_incidence_runs(
    feature_data: pd.DataFrame, feature_names, threshold=0.2
) -> pd.DataFrame:
    runs = []
    for i in range(10):
        predictor, _, __, ___, training_user_ids = train_model(
            feature_names, target, cleaned_feature_data, random_state=i
        )

        not_training_data = feature_data[
            ~feature_data["user_id"].isin(training_user_ids)
        ]

        nowcasted_incidence, predicted_incidence_column = predicted_incidence(
            predictor, not_training_data, feature_names, threshold
        )
        nowcasted_incidence["run_id"] = i
        runs.append(nowcasted_incidence[[predicted_incidence_column, "run_id", "date"]])

    predicted_incidence_runs = pd.concat(runs).reset_index().sort_values("date")

    predicted_incidence_plot_data = (
        predicted_incidence_runs.groupby("date")
        .agg({predicted_incidence_column: ["mean", "std"]})
        .reset_index()
    )

    predicted_incidence_plot_data.columns = ["date", "mean", "std"]
    return predicted_incidence_plot_data

In [ ]:
def calculate_questionnaire_incidence(
    cleaned_feature_data: pd.DataFrame,
) -> pd.DataFrame:

    res1 = cleaned_feature_data.replace(
        {True: 1, False: 0, None: 0, float("nan"): 0}
    ).reset_index()[["test_result", "date", "user_id"]]

    res2 = res1.groupby("date").agg({"test_result": ["sum", "count"]})

    res2.columns = res2.columns.droplevel(0)

    res2["rate"] = res2["sum"].div(res2["count"])

    res2["incidence"] = res2.rolling("7D")["rate"].mean() * 100_000

    questionnaire_incidence = res2.reset_index().dropna(axis=0)[["incidence", "date"]]
    questionnaire_incidence["date"] = questionnaire_incidence["date"] + timedelta(
        days=7
    )
    return questionnaire_incidence

In [ ]:
import plotly.graph_objects as go
from datetime import date, datetime

SKIP_DAYS = 5

# All
threshold = 0.6
after = cleaned_feature_data[["date"]] > pd.to_datetime(
    date(year=2021, month=11, day=1)
)
before = cleaned_feature_data[["date"]] < pd.to_datetime(
    datetime.now().date() - timedelta(days=1)
)

working_data = cleaned_feature_data[(before & after).values]

questionnaire_incidence = calculate_questionnaire_incidence(working_data)
predicted_incidence_plot_data = predict_incidence_runs(
    working_data, feature_names, threshold
)

In [ ]:
predicted_incidence_plot_data.sort_values("date", inplace=True)

default_layout = {
    "title": {"y": 0.97, "x": 0.5, "xanchor": "center", "yanchor": "top"},
    "font_size": 12,
    "autosize": True,
    "height": 600,
    "margin": dict(l=60, r=30, b=75, t=80, pad=0),
    "template": "plotly_white",
    "hovermode": "x",
    "legend": {
        "font_size": 11,
        "x": 0,
        "y": 1,
        "itemclick": "toggleothers",
        "itemdoubleclick": "toggle",
        "tracegroupgap": 2,
    },
    "hoverlabel": {"font_color": "white", "namelength": -1},
    "xaxis": {
        "showline": True,
        "linewidth": 1.5,
        "linecolor": "#455266",
        "ticks": "outside",
        "tickwidth": 1.5,
        "rangemode": "nonnegative",
    },
    "yaxis": {
        "showline": True,
        "linewidth": 1.5,
        "linecolor": "#455266",
        "ticks": "outside",
        "tickwidth": 1.5,
        "rangemode": "nonnegative",
        "range": [0, 12_000],
    },
    "showlegend": True,
}

i18n = {
    "de": {
        "booster": "Mit Auffrischimpfung 18-59 Jahre",
        "base": "Grundimmunisierte  18-59 Jahre",
        "unvax": "Ungeimpfte 18-59 Jahre",
        "title": "Gemeldete Inzidenz vs. Machine Learning Nowcast",
        "y_axis_title": "Inzidenz",
        "nowcast": "Nowcast",
        "insecurity": "Modell Unsicherheit",
    },
    "en": {
        "booster": "With booster 18-59 years",
        "base": "Vaccinated  18-59 years",
        "unvax": "Unvaccinated 18-59 years",
        "title": "Reported Incidence vs. Machine Learning Nowcast",
        "y_axis_title": "Incidence",
        "nowcast": "Nowcast",
        "insecurity": "Confidence interval",
    },
}

for language, translation in i18n.items():
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            y=predicted_incidence_plot_data["mean"].values[SKIP_DAYS:-1]
            + 2 * predicted_incidence_plot_data["std"].values[SKIP_DAYS:-1],
            x=predicted_incidence_plot_data["date"].values[SKIP_DAYS:-1],
            showlegend=False,
            legendgroup="nowcast",
            mode="lines",
            marker=dict(color="#444"),
            line=dict(width=0),
            hoverinfo="skip",
        )
    )
    fig.add_trace(
        go.Scatter(
            name=translation["insecurity"],
            y=predicted_incidence_plot_data["mean"].values[SKIP_DAYS:-1]
            - 2 * predicted_incidence_plot_data["std"].values[SKIP_DAYS:-1],
            x=predicted_incidence_plot_data["date"].values[SKIP_DAYS:-1],
            showlegend=False,
            legendgroup="nowcast",
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            fillcolor="rgba(65, 172, 221, 0.3)",  #'rgba(68, 68, 68, 0.3)'
            fill="tonexty",
            hoverinfo="skip",
        )
    )

    fig.add_trace(
        go.Scatter(
            y=predicted_incidence_plot_data["mean"].values[SKIP_DAYS:-1],
            x=predicted_incidence_plot_data["date"].values[SKIP_DAYS:-1],
            name=translation["nowcast"],
            legendgroup="nowcast",
            mode="lines",
            hovertemplate="%{y:.0f}",
            line=dict(width=2.5, color="#41ACDD"),  # 455266
        )
    )

    fig.add_trace(
        go.Scatter(
            y=questionnaire_incidence["incidence"].values[:-1],
            x=questionnaire_incidence["date"].values[:-1],
            name="questionnaire incidence",
            legendgroup="nowcast",
            mode="lines",
            hovertemplate="%{y:.0f}",
            line=dict(width=2.5, color="red"),  # 455266
        )
    )
    layout = dict(
        title_text=translation["title"],
        yaxis_title=translation["y_axis_title"],
        hovermode="x unified",
        hoverlabel={
            "font": {
                "color": "black",
            }
        },
    )
    fig.update_layout(default_layout)
    fig.update_layout(layout)
    fig.write_html(
        f"incidence_nowcast_{language}.html",
        include_plotlyjs="cdn",
        full_html=False,
        config={"responsive": "true", "displayModeBar": False},
    )

fig.write_image(f"incidence_nowcast.png")
fig.show()